In [1]:
# !git clone https://github.com/rr2n17/AdvML.git

In [2]:
!ls

test.csv  train.csv  upsampling.ipynb


## Install libraries

In [6]:
# !pip install --upgrade pip
# !pip install nltk
# !apt-get install libenchant1c2a
# !pip install pyenchant
# !pip install joblib textblob -U

In [4]:
import pandas as pd
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import word_tokenize
from random import randint
import nltk.data
from nltk.tokenize import TreebankWordTokenizer
import re
import enchant
from enchant.checker import SpellChecker
from joblib import Parallel, delayed
from textblob import TextBlob
from textblob.translate import NotTranslated

In [7]:
# nltk.download("averaged_perceptron_tagger")
# nltk.download("wordnet")
# nltk.download('punkt')
# nltk.download('stopwords')

## Read data

In [62]:
path = "./train.csv"
dataset = pd.read_csv(path)
dataset.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


## Upsample

In [63]:
ds = dataset[dataset['toxic'] == 1]
print(len(ds))
ds = ds[['comment_text', 'toxic']].iloc[:100]

tokenizer = TreebankWordTokenizer()
chkr = SpellChecker("en_UK", "en_US")

def low(x):
    # spelling checker
    x = re.sub(r'[\W_]+', ' ', x)
    chkr.set_text(x)
    for err in chkr:
        if len(err.suggest()) > 0:
            sug = err.suggest()[0]
            err.replace(sug)
            x = chkr.get_text()
    
    return x.lower()


ds['comment_text'] = ds['comment_text'].apply(lambda x: low(x))

15294


In [64]:
def translate(comment, language):
    if hasattr(comment, "decode"):
        comment = comment.decode("utf-8")

    text = TextBlob(comment)
    try:
        text = text.translate(to='de')
        text = text.translate(to=language)
        text = text.translate(to="en")
        text = re.sub(r'[\W_]+', ' ', str(text))
    except NotTranslated:
        pass

    return text

In [65]:
dp = pd.DataFrame()
df = ds[["comment_text"]].copy()

def upsample(comment):
    global dp
    tmp = []
#     l = ['es', 'fr', 'it']
    l = 'fr'
    new_comment = translate(comment, l)
#     tmp = tmp.append(new_comment)
#     dp['fr'] = pd.concat([tmp, dp], axis=1)
    return new_comment
    
        
dp['fr'] = df.apply(lambda x: upsample(x["comment_text"]), axis=1)

In [66]:
pd.DataFrame(dp)

,fr
6,Cocksucker before pissing on my job
12,Hey what is it what is it an exclusive group o...
16,Bye do not look do not come or do not think to...
42,you are gay or antisemitism Archangel meow whi...
43,Fuck your dirty mother in the ass
44,I m sorry I m sorry I m around me with someone...
51,Fuck it You re messed up I have a drink you ca...
55,Stupid peace of shit stop clearing my business...
56,Tony Sid is obviously a handful that he loves ...
58,thought deleting my page from the group I was ...


In [67]:
# Load the pretrained neural net
tokenizer = nltk.data.load('../../../nltk_data/tokenizers/punkt/english.pickle')

for text in dp["fr"]:
    output=""
    # Tokenize the text
    tokenized = tokenizer.tokenize(text)

    # Get the list of words from the entire text
    words = word_tokenize(text)

    # Identify the parts of speech
    tagged = nltk.pos_tag(words)

    for i in range(0,len(words)):
        
        replacements = []

        # Only replace nouns with nouns, vowels with vowels etc.
        for syn in wordnet.synsets(words[i]):

            # Do not attempt to replace proper nouns or determiners
            if tagged[i][1] == 'NNP' or tagged[i][1] == 'DT':
                break

            # The tokenizer returns strings like NNP, VBP etc
            # but the wordnet synonyms has tags like .n.
            # So we extract the first character from NNP ie n
            # then we check if the dictionary word has a .n. or not 
            word_type = tagged[i][1][0].lower()
            if syn.name().find("."+word_type+"."):
                # extract the word only
                r = syn.name()[0:syn.name().find(".")]
                replacements.append(r)

        if len(replacements) > 0:
            # Choose a random replacement
            replacement = replacements[randint(0,len(replacements)-1)]
            output = output + " " + replacement
        else:
            # If no replacement could be found, then just use the
            # original word
            output = output + " " + words[i]
    
    dp = dp.append(pd.Series({"fr":output}),ignore_index=True)

In [68]:
def tok_and_rem(x):
    s = x.split()
    l = list(s)
    for word in s:
        if (len(word) <= 2):
            l.remove(word)
        elif word in stopwords.words('english'):
            l.remove(word)
    return l

dp['fr'] = dp['fr'].apply(lambda x: tok_and_rem(x))

In [69]:
dp.shape

(200, 1)

In [70]:
dp.to_csv("toxic_french.csv")